## Billboard Initial Data Collection, Observation, and Cleaning

This notebook describes how data was collected from the Billboard Year-End Hot 100 charts from 2017, 2020, and 2021 using Billboard and Spotify's APIs. Data collected includes audio metrics of individual songs, artist names, and year-end rankings.

Afterwards, this dataset is observed and cleaned.

In [1]:
# Importing modules
import billboard
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import time
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Collecting year-end charts
year_end_2017 = billboard.ChartData('hot-100-songs', year = 2017)
year_end_2020 = billboard.ChartData('hot-100-songs', year = 2020)
year_end_2021 = billboard.ChartData('hot-100-songs', year = 2021)

In [3]:
# Transforming 2017 chart into DataFrame
song_list_2017 = []
for song in year_end_2017:
    song_list_2017.append([song.title, song.artist, song.rank])
song_data_2017 = pd.DataFrame(song_list_2017, columns = ['Title', 'Artist', 'Rank'])
song_data_2017

,Title,Artist,Rank
0,Shape Of You,Ed Sheeran,1
1,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber,2
2,That's What I Like,Bruno Mars,3
3,Humble.,Kendrick Lamar,4
4,Something Just Like This,The Chainsmokers & Coldplay,5
...,...,...,...
95,Havana,Camila Cabello Featuring Young Thug,96
96,What Lovers Do,Maroon 5 Featuring SZA,97
97,Do Re Mi,blackbear,98
98,Look At Me!,XXXTENTACION,99


In [4]:
# Transforming 2020 chart into DataFrame
song_list_2020 = []
for song in year_end_2020:
    song_list_2020.append([song.title, song.artist, song.rank])
song_data_2020 = pd.DataFrame(song_list_2020, columns = ['Title', 'Artist', 'Rank'])
song_data_2020

,Title,Artist,Rank
0,Blinding Lights,The Weeknd,1
1,Circles,Post Malone,2
2,The Box,Roddy Ricch,3
3,Don't Start Now,Dua Lipa,4
4,Rockstar,DaBaby Featuring Roddy Ricch,5
...,...,...,...
95,More Than My Hometown,Morgan Wallen,96
96,Lovin' On You,Luke Combs,97
97,Said Sum,Moneybagg Yo,98
98,Slide,H.E.R. Featuring YG,99


In [5]:
# Transforming 2021 chart into DataFrame
song_list_2021 = []
for song in year_end_2021:
    song_list_2021.append([song.title, song.artist, song.rank])
song_data_2021 = pd.DataFrame(song_list_2021, columns = ['Title', 'Artist', 'Rank'])
song_data_2021

,Title,Artist,Rank
0,Levitating,Dua Lipa,1
1,Save Your Tears,The Weeknd & Ariana Grande,2
2,Blinding Lights,The Weeknd,3
3,Mood,24kGoldn Featuring iann dior,4
4,Good 4 U,Olivia Rodrigo,5
...,...,...,...
95,Things A Man Oughta Know,Lainey Wilson,96
96,Throat Baby (Go Baby),BRS Kash,97
97,Tombstone,Rod Wave,98
98,Drinkin' Beer. Talkin' God. Amen.,Chase Rice Featuring Florida Georgia Line,99


In [6]:
# Locating Spotify client ID and secret
cid = 'CLIENT ID'
secret = 'SECRET|'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [7]:
# Collecting track IDs based on playlist URL
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids_2017 = getTrackIDs('spotify', '2XPEN88QyrPQ9zGqS8uS2x')
ids_2020 = getTrackIDs('spotify', '1WBljFutuk7uLQtfqfmjWV')
ids_2021 = getTrackIDs('spotify', '5Nt7KFSEfXIlsDIB8SCpNU')

In [8]:
# Collecting track features for each song
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  duration_ms = meta['duration_ms']
  popularity = meta['popularity']
  explicit = meta['explicit']
    
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']


  track = [name, album, artist, release_date, duration_ms, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence, explicit]
  return track

In [9]:
# Looping over 2017 track ids to append track-level metrics in a new row and create a dataset
tracks_2017 = []
for i in range(len(ids_2017)):
  time.sleep(.5)
  track = getTrackFeatures(ids_2017[i])
  tracks_2017.append(track)
tracks_2017_df = pd.DataFrame(tracks_2017, columns = ['name', 'album', 'artist', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'explicit'])
tracks_2017_df

,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,Shape of You,÷ (Deluxe),Ed Sheeran,2017-03-03,233712,88,0.825,0.581000,0.825,0.652,0.000000,0.0931,-3.183,0.0802,95.977,4,0.931,False
1,Despacito - Remix,Despacito Feat. Justin Bieber (Remix),Luis Fonsi,2017-04-17,228826,75,0.653,0.228000,0.653,0.816,0.000000,0.0967,-4.353,0.1670,178.085,4,0.816,False
2,That's What I Like,24K Magic,Bruno Mars,2016-11-17,206693,84,0.853,0.013000,0.853,0.560,0.000000,0.0944,-4.961,0.0406,134.066,4,0.860,False
3,HUMBLE.,DAMN.,Kendrick Lamar,2017-04-14,177000,85,0.908,0.000282,0.908,0.621,0.000054,0.0958,-6.638,0.1020,150.011,4,0.421,True
4,Something Just Like This,Memories...Do Not Open,The Chainsmokers,2017-04-07,247160,86,0.617,0.049800,0.617,0.635,0.000014,0.1640,-6.769,0.0317,103.019,4,0.446,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Havana (feat. Young Thug),Havana (feat. Young Thug),Camila Cabello,2017-08-03,216896,2,0.768,0.186000,0.768,0.517,0.000038,0.1040,-4.323,0.0312,104.992,4,0.418,False
96,What Lovers Do (feat. SZA),Red Pill Blues (Deluxe),Maroon 5,2017-11-03,199849,0,0.792,0.080500,0.792,0.612,0.000004,0.0852,-5.212,0.0693,109.959,4,0.420,False
97,do re mi,digital druglord,blackbear,2017-04-21,212027,38,0.745,0.005220,0.745,0.593,0.000005,0.1230,-6.350,0.0526,111.002,3,0.170,True
98,Look At Me!,Look At Me!,XXXTENTACION,2017-02-20,126345,85,0.763,0.259000,0.763,0.726,0.000000,0.0976,-6.405,0.2820,139.059,4,0.349,True


In [10]:
# Looping over 2020 track ids to append track-level metrics in a new row and create a dataset
tracks_2020 = []
for i in range(len(ids_2020)):
  time.sleep(.5)
  track = getTrackFeatures(ids_2020[i])
  tracks_2020.append(track)
tracks_2020_df = pd.DataFrame(tracks_2020, columns = ['name', 'album', 'artist', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'explicit'])
tracks_2020_df

,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,Blinding Lights,Blinding Lights,The Weeknd,2019-11-29,201573,22,0.513,0.00147,0.513,0.796,0.000209,0.0938,-4.075,0.0629,171.017,4,0.345,False
1,Circles,Hollywood's Bleeding,Post Malone,2019-09-06,215280,89,0.695,0.19200,0.695,0.762,0.002440,0.0863,-3.497,0.0395,120.042,4,0.553,False
2,The Box,Please Excuse Me for Being Antisocial,Roddy Ricch,2019-12-06,196652,86,0.896,0.10400,0.896,0.586,0.000000,0.7900,-6.687,0.0559,116.971,4,0.642,True
3,Don't Start Now,Don't Start Now,Dua Lipa,2019-10-31,183290,84,0.794,0.01250,0.794,0.793,0.000000,0.0952,-4.521,0.0842,123.941,4,0.677,False
4,ROCKSTAR (feat. Roddy Ricch),BLAME IT ON BABY,DaBaby,2020-04-17,181733,86,0.746,0.24700,0.746,0.690,0.000000,0.1010,-7.956,0.1640,89.977,4,0.497,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,More Than My Hometown,More Than My Hometown,Morgan Wallen,2020-04-17,216573,64,0.621,0.60100,0.621,0.882,0.000000,0.1320,-5.010,0.0459,126.014,4,0.574,False
96,Lovin' On You,What You See Is What You Get,Luke Combs,2019-11-08,194866,72,0.572,0.00165,0.572,0.949,0.000195,0.1630,-4.865,0.0600,118.974,4,0.530,False
97,Said Sum,Code Red,Moneybagg Yo,2020-09-18,155168,72,0.929,0.01850,0.929,0.667,0.000000,0.1000,-6.789,0.3530,126.998,4,0.274,True
98,Slide,Slide,H.E.R.,2019-09-27,238321,70,0.832,0.08070,0.832,0.469,0.000008,0.2070,-9.141,0.3390,97.023,4,0.197,True


In [11]:
# Looping over 2021 track ids to append track-level metrics in a new row and create a dataset
tracks_2021 = []
for i in range(len(ids_2021)):
  time.sleep(.5)
  track = getTrackFeatures(ids_2021[i])
  tracks_2021.append(track)
tracks_2021_df = pd.DataFrame(tracks_2021, columns = ['name', 'album', 'artist', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'explicit'])
tracks_2021_df

,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,Levitating,Future Nostalgia,Dua Lipa,2020-03-27,203807,87,0.695,0.05610,0.695,0.884,0.000000,0.2130,-2.278,0.0753,103.014,4,0.914,False
1,Save Your Tears,After Hours,The Weeknd,2020-03-20,215626,91,0.680,0.02120,0.680,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4,0.644,True
2,Blinding Lights,After Hours,The Weeknd,2020-03-20,200040,94,0.514,0.00146,0.514,0.730,0.000095,0.0897,-5.934,0.0598,171.005,4,0.334,False
3,Mood (feat. iann dior),El Dorado,24kGoldn,2021-03-26,140533,90,0.701,0.17400,0.701,0.716,0.000000,0.3240,-3.671,0.0361,91.007,4,0.732,True
4,good 4 u,good 4 u,Olivia Rodrigo,2021-05-14,178147,40,0.556,0.30000,0.556,0.661,0.000000,0.1010,-5.052,0.2040,168.560,4,0.668,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Things A Man Oughta Know,Sayin' What I'm Thinkin',Lainey Wilson,2021-02-19,203373,73,0.659,0.51300,0.659,0.683,0.000005,0.1330,-5.623,0.0312,139.931,4,0.397,False
96,Throat Baby (Go Baby),Throat Baby (Go Baby),BRS Kash,2020-07-24,218181,60,0.900,0.01350,0.900,0.521,0.000000,0.0951,-7.286,0.1470,132.007,4,0.213,True
97,Tombstone,SoulFly,Rod Wave,2021-03-26,160078,74,0.550,0.59700,0.550,0.637,0.000000,0.1290,-5.212,0.1630,84.448,4,0.535,True
98,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,Chase Rice,2020-11-30,160839,68,0.627,0.18500,0.627,0.678,0.000000,0.3740,-4.691,0.0294,100.032,4,0.724,False


In [12]:
# Merging 2017 Spotify metrics dataset with Billboard chart performance dataset and exporting to CSV
charts_merged_2017 = pd.concat([song_data_2017, tracks_2017_df], axis = 1)
charts_merged_2017
charts_merged_2017.to_csv("../../data/Raw/billboard2017.csv", sep = ',')

In [13]:
# Merging 2020 Spotify metrics dataset with Billboard chart performance dataset and exporting to CSV
charts_merged_2020 = pd.concat([song_data_2020, tracks_2020_df], axis = 1)
charts_merged_2020
charts_merged_2020.to_csv("../../data/Raw/billboard2020.csv", sep = ',')

In [14]:
# Merging 2021 Spotify metrics dataset with Billboard chart performance dataset and exporting to CSV
charts_merged_2021 = pd.concat([song_data_2021, tracks_2021_df], axis = 1)
charts_merged_2021
charts_merged_2021.to_csv("../../data/Raw/billboard2021.csv", sep = ',')

#### Data Observation

In [15]:
# Loading in 2017 data from CSV
billboard2017_df = pd.read_csv("../../data/Raw/billboard2017.csv")
billboard2017_df.head()

,Unnamed: 0,Title,Artist,Rank,name,album,artist,release_date,duration_ms,popularity,...,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,0,Shape Of You,Ed Sheeran,1,Shape of You,÷ (Deluxe),Ed Sheeran,2017-03-03,233712,88,...,0.825,0.652,0.000000,0.0931,-3.183,0.0802,95.977,4,0.931,False
1,1,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bieber,2,Despacito - Remix,Despacito Feat. Justin Bieber (Remix),Luis Fonsi,2017-04-17,228826,75,...,0.653,0.816,0.000000,0.0967,-4.353,0.1670,178.085,4,0.816,False
2,2,That's What I Like,Bruno Mars,3,That's What I Like,24K Magic,Bruno Mars,2016-11-17,206693,84,...,0.853,0.560,0.000000,0.0944,-4.961,0.0406,134.066,4,0.860,False
3,3,Humble.,Kendrick Lamar,4,HUMBLE.,DAMN.,Kendrick Lamar,2017-04-14,177000,85,...,0.908,0.621,0.000054,0.0958,-6.638,0.1020,150.011,4,0.421,True
4,4,Something Just Like This,The Chainsmokers & Coldplay,5,Something Just Like This,Memories...Do Not Open,The Chainsmokers,2017-04-07,247160,86,...,0.617,0.635,0.000014,0.1640,-6.769,0.0317,103.019,4,0.446,False


In [16]:
# Loading in 2020 data from CSV
billboard2020_df = pd.read_csv("../../data/Raw/billboard2020.csv")
billboard2020_df.head()

,Unnamed: 0,Title,Artist,Rank,name,album,artist,release_date,duration_ms,popularity,...,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,0,Blinding Lights,The Weeknd,1,Blinding Lights,Blinding Lights,The Weeknd,2019-11-29,201573,22,...,0.513,0.796,0.000209,0.0938,-4.075,0.0629,171.017,4,0.345,False
1,1,Circles,Post Malone,2,Circles,Hollywood's Bleeding,Post Malone,2019-09-06,215280,89,...,0.695,0.762,0.002440,0.0863,-3.497,0.0395,120.042,4,0.553,False
2,2,The Box,Roddy Ricch,3,The Box,Please Excuse Me for Being Antisocial,Roddy Ricch,2019-12-06,196652,86,...,0.896,0.586,0.000000,0.7900,-6.687,0.0559,116.971,4,0.642,True
3,3,Don't Start Now,Dua Lipa,4,Don't Start Now,Don't Start Now,Dua Lipa,2019-10-31,183290,84,...,0.794,0.793,0.000000,0.0952,-4.521,0.0842,123.941,4,0.677,False
4,4,Rockstar,DaBaby Featuring Roddy Ricch,5,ROCKSTAR (feat. Roddy Ricch),BLAME IT ON BABY,DaBaby,2020-04-17,181733,86,...,0.746,0.690,0.000000,0.1010,-7.956,0.1640,89.977,4,0.497,True


In [17]:
# Loading in 2021 data from CSV
billboard2021_df = pd.read_csv("../../data/Raw/billboard2021.csv")
billboard2021_df.head()

,Unnamed: 0,Title,Artist,Rank,name,album,artist,release_date,duration_ms,popularity,...,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,0,Levitating,Dua Lipa,1,Levitating,Future Nostalgia,Dua Lipa,2020-03-27,203807,87,...,0.695,0.884,0.000000,0.2130,-2.278,0.0753,103.014,4,0.914,False
1,1,Save Your Tears,The Weeknd & Ariana Grande,2,Save Your Tears,After Hours,The Weeknd,2020-03-20,215626,91,...,0.680,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4,0.644,True
2,2,Blinding Lights,The Weeknd,3,Blinding Lights,After Hours,The Weeknd,2020-03-20,200040,94,...,0.514,0.730,0.000095,0.0897,-5.934,0.0598,171.005,4,0.334,False
3,3,Mood,24kGoldn Featuring iann dior,4,Mood (feat. iann dior),El Dorado,24kGoldn,2021-03-26,140533,90,...,0.701,0.716,0.000000,0.3240,-3.671,0.0361,91.007,4,0.732,True
4,4,Good 4 U,Olivia Rodrigo,5,good 4 u,good 4 u,Olivia Rodrigo,2021-05-14,178147,40,...,0.556,0.661,0.000000,0.1010,-5.052,0.2040,168.560,4,0.668,True


In [18]:
# Examining DataFrame dimensions
print(billboard2017_df.shape, billboard2020_df.shape, billboard2021_df.shape)

(100, 22) (100, 22) (100, 22)


All datasets contain data from the Top 100 Billboard Year-End Songs from the years 2017, 2020, and 2021 and currently 22 attributes are being tracked. However, it may be worthwhile to clean up the data before we do preliminary analysis. 

In [19]:
billboard2017_df.columns

Index(['Unnamed: 0', 'Title', 'Artist', 'Rank', 'name', 'album', 'artist',
       'release_date', 'duration_ms', 'popularity', 'danceability',
       'acousticness', 'danceability.1', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature',
       'valence', 'explicit'],
      dtype='object')

'Title' and 'name' are redundant columns, as are 'Artist' and 'artist'. Additionally, the columns 'Unnamed: 0' and 'danceability.1' appear to be extraneous. I also like what Srinidhi did to convert 'duration_ms' to seconds. I'm going to make new datasets with this all cleaned up, using similar cleaning processes as were performed on the Spotify datasets. 

#### Data Cleaning

In [20]:
## 2017 Billboard Data Cleaning ##
# Dropping extraneous columns
billboard2017_clean = billboard2017_df.drop(['Title', 'Artist', 'Unnamed: 0', 'danceability.1'], axis = 1)

# Converting and creating duration column
billboard2017_clean['duration_sec'] = billboard2017_df['duration_ms']/1000

# Export cleaned data to CSV
billboard2017_clean.to_csv('../../data/Clean/billboard2017cleaned.csv', sep = ',')

billboard2017_clean.sample()

,Rank,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit,duration_sec
48,49,CAN'T STOP THE FEELING! (from DreamWorks Anima...,CAN'T STOP THE FEELING! (from DreamWorks Anima...,Justin Timberlake,2016-05-06,236001,80,0.666,0.0123,0.83,0.0,0.191,-5.715,0.0751,113.03,4,0.702,False,236.001


In [21]:
## 2020 Billboard Data Cleaning ##
# Dropping extraneous columns
billboard2020_clean = billboard2020_df.drop(['Title', 'Artist', 'Unnamed: 0', 'danceability.1'], axis = 1)

# Converting and creating duration column
billboard2020_clean['duration_sec'] = billboard2020_df['duration_ms']/1000

# Export cleaned data to CSV
billboard2020_clean.to_csv('../../data/Clean/billboard2020cleaned.csv', sep = ',')

billboard2020_clean.sample()

,Rank,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit,duration_sec
21,22,Falling,Falling,Trevor Daniel,2018-10-05,159381,40,0.785,0.123,0.431,0.0,0.0887,-8.756,0.0364,127.085,4,0.236,False,159.381


In [22]:
## 2021 Billboard Data Cleaning ##
# Dropping extraneous columns
billboard2021_clean = billboard2021_df.drop(['Title', 'Artist', 'Unnamed: 0', 'danceability.1'], axis = 1)

# Converting and creating duration column
billboard2021_clean['duration_sec'] = billboard2021_df['duration_ms']/1000

# Export cleaned data to CSV
billboard2021_clean.to_csv('../../data/Clean/billboard2021cleaned.csv', sep = ',')

billboard2021_clean.sample()

,Rank,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit,duration_sec
40,41,Dynamite,BE,BTS,2020-11-20,199053,87,0.746,0.0112,0.765,0.0,0.0936,-4.41,0.0993,114.044,4,0.737,False,199.053
